In [0]:
import pandas as pd
import numpy as np
import warnings
import os
import tensorflow as tf

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!cd 'gdrive/My Drive/Colab Notebooks'; ls;

 02_data_preprocessing_solution.ipynb
 06_decision_trees_solution.ipynb
 12_deep_neural_networks.ipynb
 13_convolutional_neural_networks_solution.ipynb
 2017년_가구에너지_상설표본조사_변수설명.xlsx
 2017년_가구에너지_상설표본조사_마이크로데이터.xlsx
 COMEX_Copper_daily.csv
 COMEX_Copper_daily.txt
 COMEX_Gold_daily.csv
 COMEX_Gold_daily.txt
 gbst_importance.xlsx
 HH_daily.csv
 HH_daily.txt
 HH_daily.xls
 노윤곤_중간과제.ipynb
 listings.csv
 listings_real.csv
 LME_Copper_daily.csv
 LME_Copper_daily.txt
 logs
 Nickel_daily.csv
 Nickel_daily.txt
 노윤곤_PDF.pdf
 review_en.csv
 reviews.csv
 rnd_best.dot
 rnd_importance.xlsx
 saved_model
 sklearn_LDA.ipynb
'Structural Break ANN_keras.ipynb'
 Structural_Break_ANN_nickel_deep_dropout_batch_normalization.ipynb
 test
'transformer.ipynb의 사본'
 Untitled
 Untitled0.ipynb
 WTI_daily.csv
 WTI_daily.txt
 WTI_daily.xls
 Zinc_daily.csv
 Zinc_daily.txt


In [0]:
df1 = pd.read_csv('gdrive/My Drive/Colab Notebooks/listings.csv')
df2 = pd.read_csv('gdrive/My Drive/Colab Notebooks/reviews.csv')

In [5]:
df1.columns[:20]

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id'],
      dtype='object')

In [6]:
df1.columns[20:40]

Index(['host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed'],
      dtype='object')

In [7]:
df1.columns[40:60]

Index(['neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
       'square_feet'],
      dtype='object')

In [8]:
df1.columns[60:80]

Index(['price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90'],
      dtype='object')

In [9]:
df1.columns[80:100]

Index(['availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'license',
       'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture'],
      dtype='object')

In [10]:
df1.columns[100:]

Index(['require_guest_phone_verification', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')

In [11]:
df2.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [12]:
len(df1)

50796

In [13]:
len(df2)

1273976

In [14]:
df1['price']

0          $100.00
1          $225.00
2           $89.00
3          $200.00
4           $94.00
           ...    
50791      $150.00
50792      $149.00
50793      $188.00
50794    $1,100.00
50795      $450.00
Name: price, Length: 50796, dtype: object

In [0]:
attr = ['id','host_response_rate','host_is_superhost','host_listings_count','room_type','accommodates','bedrooms',
        'review_scores_rating','latitude','longitude','price']

df = df1[attr]

In [0]:
df.dropna(subset=['review_scores_rating','price'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [0]:
df2.drop(['id', 'reviewer_id', 'reviewer_name','date'], axis=1, inplace=True)
df2.dropna(subset=['comments'],inplace=True)
df2 = df2[~df2.comments.str.contains('This is an automated posting.')]
df2.reset_index(drop=True,inplace=True)

In [18]:
!pip install pycld2

In [19]:
from tqdm import tqdm_notebook as tqdm
import pycld2 as cld2
tqdm().pandas()

i = -1
lang = []

for comments in tqdm(df2['comments']):
  i += 1
  try:
    isReliable, textBytesFound, details = cld2.detect(comments)
    lang.append(details[0][0])
  except:
    lang.append('N')

df2['lang'] = lang

In [0]:
df2 = df2.loc[df2['lang'] == 'ENGLISH']
df2.drop(['lang'], axis=1, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [21]:
df

,id,host_response_rate,host_is_superhost,host_listings_count,room_type,accommodates,bedrooms,review_scores_rating,latitude,longitude,price
0,2060,22%,f,0.0,Private room,2,1.0,80.0,40.85722,-73.93790,$100.00
1,2595,87%,f,6.0,Entire home/apt,2,0.0,94.0,40.75362,-73.98377,$225.00
2,3831,83%,f,1.0,Entire home/apt,3,1.0,90.0,40.68514,-73.95976,$89.00
3,5099,NaN,f,1.0,Entire home/apt,2,1.0,90.0,40.74767,-73.97500,$200.00
4,5114,50%,f,3.0,Private room,2,1.0,94.0,40.71910,-73.99040,$94.00
...,...,...,...,...,...,...,...,...,...,...,...
39360,42750206,NaN,f,0.0,Private room,2,1.0,100.0,40.74024,-73.82093,$39.00
39361,42762152,100%,f,1.0,Private room,2,1.0,100.0,40.68645,-73.93803,$80.00
39362,42794256,NaN,f,0.0,Private room,2,1.0,100.0,40.83212,-73.94180,$55.00
39363,42816035,100%,f,10.0,Entire home/apt,3,2.0,100.0,40.76166,-73.99113,$180.00


In [22]:
df2

,listing_id,comments
0,2060,"very nice neighborhood,close enough to ""A"" tra..."
1,2595,I've stayed with my friend at the Midtown Cast...
2,2595,"We've been staying here for about 9 nights, en..."
3,2595,We had a wonderful stay at Jennifer's charming...
4,2595,Hi to everyone!\r\nWould say our greatest comp...
...,...,...
1099489,42762152,Great place to stay. Easily accessible from pu...
1099490,42794256,"As a host myself, I am impressed with Andre an..."
1099491,42794256,Great place that hit the mark for my night in ...
1099492,42816035,This duplex home is a real gem in New York Cit...


In [0]:
df.to_csv('gdrive/My Drive/Colab Notebooks/listings_cleaned.csv')
df2.to_csv('gdrive/My Drive/Colab Notebooks/reviews_cleaned.csv')

In [0]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/listings_cleaned.csv')
df2 = pd.read_csv('gdrive/My Drive/Colab Notebooks/reviews_cleaned.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
import tensorflow_dataset as tfds
from tf.keras.preprocessing.sequence import pad_sequences

tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(df2['comments'], target_vocab_size=2**13)
df2_tokenized = tokenizer.encode(df2['comments'])
df2_padded = pad_sequences(df2_tokenized, maxlen=200)

In [0]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df[attr,'comments'].values, tf.float32),
            tf.cast(df['price'].values, tf.int32)
        )
    )
)

for features_tensor, target_tensor in training_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')

In [0]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
df.set_index('id', inplace=True)
df2.set_index('listing_id',inplace=True)

X_train = df2.join(df,how='inner')

In [0]:
len(X_train)

967828

In [0]:
X_train.drop(['host_response_rate'], axis=1, inplace=True)
X_train.drop(['property_type'], axis=1, inplace=True)
X_train.drop(['date'], axis=1, inplace=True)
X_train.reset_index(drop=True, inplace=True)

In [0]:
X_train = X_train.join(train_vector)
X_train.head()

,comments,host_is_superhost,host_listings_count,accommodates,bedrooms,review_scores_rating,latitude,longitude,reviews_per_month,price,...,90,91,92,93,94,95,96,97,98,99
0,Great host,f,6.0,2,1.0,98.0,40.64749,-73.97237,0.21,$149.00,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,Nice room for the price. Great neighborhood. J...,f,6.0,2,1.0,98.0,40.64749,-73.97237,0.21,$149.00,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,Very nice apt. New remodeled.,f,6.0,2,1.0,98.0,40.64749,-73.97237,0.21,$149.00,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,Great place to stay for a while. John is a gre...,f,6.0,2,1.0,98.0,40.64749,-73.97237,0.21,$149.00,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,I really enjoyed my time here in deep south Br...,f,6.0,2,1.0,98.0,40.64749,-73.97237,0.21,$149.00,...,0.0,0.0,0.175647,0.232726,0.0,0.199682,0.0,0.0,0.0,0.0


In [0]:
del df2, dataset, train_vector

In [0]:
import re

X_train['price'] = re.sub('[^0-9,]', '', X_train['price'])
X_train.price = X_train.price.astype('float32')

X_train = X_train[X_train['price'].str.contains('{')==False]
X_train = X_train[X_train['price'].str.contains('t')==False]
X_train = X_train[X_train['price'].str.contains('f')==False]
X_train = X_train[X_train['price'].str.contains('a')==False]
X_train = X_train[X_train['price'].str.contains('e')==False]
X_train = X_train[X_train['price'].str.contains('i')==False]
X_train = X_train[X_train['price'].str.contains('o')==False]
X_train = X_train[X_train['price'].str.contains('u')==False]
X_train = X_train[X_train['price'].str.contains('n')==False]
X_train = X_train[X_train['price'].str.contains('녽')==False]
X_train = X_train[X_train['price'].str.contains('US')==False]
X_train['price'] = X_train['price'].map(lambda x: x.lstrip('$'))
X_train['price'] = X_train['price'].map(lambda x: x.replace(',',''))
X_train.price = X_train.price.astype('float32')

In [0]:
X_train['host_is_superhost'] = (X_train['host_is_superhost'] == 't').astype(int)

In [0]:
X_train = X_train[X_train['price'] > 0]
X_train = X_train[X_train['price'] < 3000]

In [0]:
X_train = X_train.iloc[::10, :]

In [0]:
X_train.dropna(inplace=True)

In [0]:
len(X_train)

96665

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_forecast = train_test_split(X_train, test_size=0.2, random_state=42)

In [0]:
y_train = X_train['price'].values
X_train.drop(['price'], axis=1, inplace=True)
y_forecast = X_forecast['price'].values
X_forecast.drop(['price'], axis=1, inplace=True)

In [0]:
X_train.drop(['comments'], axis=1, inplace=True)
X_forecast.drop(['comments'], axis=1, inplace=True)

In [0]:
X_train

,host_is_superhost,host_listings_count,accommodates,bedrooms,review_scores_rating,latitude,longitude,reviews_per_month,0,1,...,90,91,92,93,94,95,96,97,98,99
247235,1,1.0,4,1.0,95.0,40.68180,-73.96238,1.40,0.187724,0.0,...,0.181858,0.000000,0.000000,0.000000,0.0,0.0,0.366388,0.0,0.000000,0.000000
281005,0,1.0,4,2.0,96.0,40.76751,-73.96309,1.11,0.000000,0.0,...,0.326448,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
129730,0,1.0,2,1.0,86.0,40.79820,-73.96394,1.75,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
949721,0,1.0,2,1.0,100.0,40.70406,-73.92578,2.70,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
367856,0,1.0,2,1.0,91.0,40.82094,-73.95622,0.31,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62880,1,1.0,3,1.0,95.0,40.70462,-73.92280,0.45,0.000000,0.0,...,0.000000,0.175113,0.138122,0.183006,0.0,0.0,0.000000,0.0,0.000000,0.506985
549664,1,1.0,2,1.0,97.0,40.69849,-73.92300,4.54,0.373997,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
769313,0,7.0,2,1.0,93.0,40.74820,-73.81656,3.59,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
8800,1,2.0,6,2.0,95.0,40.82748,-73.95069,1.95,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [0]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_forecast_scaled = scaler.transform(X_forecast)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import model_from_json
from tensorflow.keras.losses import MAPE
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler
from optuna.integration import TFKerasPruningCallback

In [0]:
folder = './saved_model/ANN'
try:
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
except Exception as e:
    print(e)

def objective(trial):
    
    reset_graph()
    model = Sequential()
    
    model.add(Dense(units=50, kernel_initializer='he_normal', activation='relu', 
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=trial.suggest_uniform('rate1',0,0.5)))
    
    model.add(Dense(units=30, kernel_initializer='he_normal', activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(units=10, kernel_initializer='he_normal', activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(units=100, kernel_initializer='he_normal', activation='relu', 
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=trial.suggest_uniform('rate2',0,0.5)))
    
    model.add(Dense(units=1, kernel_initializer='he_normal', activation='linear'))
    
    optimizer = Adam(lr=trial.suggest_loguniform('lr',0.001,0.1), beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer)
    
    early_stop = EarlyStopping(monitor='val_loss',patience=20,verbose=1,restore_best_weights=True)
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    model.fit(X_train_scaled,y_train,epochs=1000,batch_size=50,verbose=0,validation_split=0.3,callbacks=[early_stop,pruning_callback])
    
    model_json = model.to_json()
    with open("./saved_model/ANN/model_{}.json".format(trial.number),"w") as json_file:
        json_file.write(model_json)
    model.save_weights("./saved_model/ANN/model_{}.h5".format(trial.number))
    
    forecast_result=model.predict(X_forecast_scaled)
    
    y_predict = np.zeros(len(y_forecast))
    
    for i in range(len(forecast_result)):
        y_predict[i] = forecast_result[i]
        
    mape1 = np.abs((y_forecast - y_predict)/(y_forecast+0.00000001))
    mape2 = 0
    
    for i in range(len(mape1)):
        mape2 += mape1[i]
        
    mape3 = 100 * (mape2 / len(mape1))
    
    return mape3

sampler = TPESampler(**TPESampler.hyperopt_parameters(), seed=42)
pruner = SuccessiveHalvingPruner(min_resource=1, reduction_factor=2, min_early_stopping_rate=1)
study = optuna.create_study(pruner=pruner, sampler=sampler)
study.optimize(objective, n_trials=30, n_jobs=1)

print(study.best_params)

Restoring model weights from the end of the best epoch.
Epoch 00081: early stopping


[I 2019-12-16 11:20:08,957] Finished trial#0 resulted in value: 28.15072372705268. Current best value is 28.15072372705268 with parameters: {'rate1': 0.18727005942368125, 'rate2': 0.4753571532049581, 'lr': 0.05395030966670229}.
[I 2019-12-16 11:20:32,370] Setting status of trial#1 as TrialState.PRUNED. Trial was pruned at epoch 2.
[I 2019-12-16 11:20:56,169] Setting status of trial#2 as TrialState.PRUNED. Trial was pruned at epoch 2.
[I 2019-12-16 11:21:20,402] Setting status of trial#3 as TrialState.PRUNED. Trial was pruned at epoch 2.
[I 2019-12-16 11:25:24,030] Setting status of trial#4 as TrialState.PRUNED. Trial was pruned at epoch 32.
[I 2019-12-16 11:26:01,821] Setting status of trial#5 as TrialState.PRUNED. Trial was pruned at epoch 4.
[I 2019-12-16 11:26:39,780] Setting status of trial#6 as TrialState.PRUNED. Trial was pruned at epoch 4.


Restoring model weights from the end of the best epoch.
Epoch 00045: early stopping


[I 2019-12-16 11:32:15,853] Finished trial#7 resulted in value: 27.057785370671. Current best value is 27.057785370671 with parameters: {'rate1': 0.06974693032602092, 'rate2': 0.14607232426760908, 'lr': 0.023246728489504348}.


Restoring model weights from the end of the best epoch.
Epoch 00053: early stopping


...

Restoring model weights from the end of the best epoch.
Epoch 00047: early stopping


[I 2019-12-16 12:10:07,471] Finished trial#22 resulted in value: 26.69319706894577. Current best value is 26.481308666791637 with parameters: {'rate1': 0.050189852859302365, 'rate2': 0.2533071298521252, 'lr': 0.010245080746773516}.


Restoring model weights from the end of the best epoch.
Epoch 00061: early stopping


[I 2019-12-16 12:16:57,783] Finished trial#23 resulted in value: 26.40091241490375. Current best value is 26.40091241490375 with parameters: {'rate1': 0.003701514446273653, 'rate2': 0.21043515131429658, 'lr': 0.01095729954358869}.
[I 2019-12-16 12:24:25,891] Setting status of trial#24 as TrialState.PRUNED. Trial was pruned at epoch 64.
[I 2019-12-16 12:25:02,397] Setting status of trial#25 as TrialState.PRUNED. Trial was pruned at epoch 4.
[I 2019-12-16 12:26:07,757] Setting status of trial#26 as TrialState.PRUNED. Trial was pruned at epoch 8.
[I 2019-12-16 12:26:45,735] Setting status of trial#27 as TrialState.PRUNED. Trial was pruned at epoch 4.
[I 2019-12-16 12:27:53,991] Setting status of trial#28 as TrialState.PRUNED. Trial was pruned at epoch 8.
[I 2019-12-16 12:29:01,417] Setting status of trial#29 as TrialState.PRUNED. Trial was pruned at epoch 8.


{'rate1': 0.003701514446273653, 'rate2': 0.21043515131429658, 'lr': 0.01095729954358869}


In [0]:
json_file = open("./saved_model/ANN/model_{}.json".format(study.best_trial.number),"r")
loaded_model_json = json_file.read()
json_file.close()
best_model = model_from_json(loaded_model_json)
best_model.load_weights("./saved_model/ANN/model_{}.h5".format(study.best_trial.number))
forecast_result=best_model.predict(X_forecast_scaled)